# Stomata Camera APIv2 - getting thermal image example

You can see the Swagger documentation here: https://app.sigrow.com/api/v2/

We will go through some basic operations regarding the Stomata Camera functionality.


In [1]:
import requests
import json
import numpy as np
import datetime

Make sure that in your requests you use the correct `X-API-Key` header:

In [2]:
X_API_KEY = open('/path/to/your/apikey/apik.txt','r').readline()

In [3]:
# If you have only one camera - this will do the trick to get camera id
selected_camera = requests.get("http://app.sigrow.com/api/v2/cameras", 
                                headers={"X-API-Key": X_API_KEY}).json()[0]['id']

In [4]:
# You can view your camera id
selected_camera

## Getting an saving the available shots for the Camera with they original filename

Let's say we want to save all images captured between 2021-10-11 19:15 and 2021-10-17 19:00. Bear in mind the time in the query should be in UTC! (See the API documentation)

In [5]:
# One of the ways to define starting date from when you want to get your images
# Scrip will iterate day by day bact to past to get images
# date is defined as year, month, day, hour, minute, second
stepis=datetime.datetime(2021, 10, 17, 23, 59, 59, 0)

In [6]:
# Define helper functions to filter only desired image
def get_high_res(item):
            return item["height"] == 600 and item["width"] == 800
    
def get_temperature_plus_high_res_processed(item):
    return get_high_res(item) and item["type"] == "THERMAL_PLUS_PNG"

In [7]:
# Define path, where you want to save your thermal images
path='/path/to/your/image/folder/with/thermal/images'

In [8]:
# Now define how many days to the past you want to get images from
# place count of days as second number
for day in range(0,6,1):
    
    # Day step adjusting is done here
    van=stepis - datetime.timedelta(days=(day+1))
    tot=stepis - datetime.timedelta(days=day)
    
    # For convenience you can print our date range from where the images are acquired
    print('Images are acquired from: {}  to: {}'.format(noo,too))
    
    # Get available shots at that day
    shots_response = requests.get("http://app.sigrow.com/api/v2/camera/{}/shots?after={}&before={}".format(
                        selected_camera, van.strftime("%Y%m%dT%H%M%SZ"), tot.strftime("%Y%m%dT%H%M%SZ")
                        ), headers={
                                    "X-API-Key": X_API_KEY
                                    }).json()
    
    # Iterate over each shot
    for num in range(len(shots_response["shots"])):
        
        # Get shot by its number
        selected_shot = shots_response["shots"][num]
        shot_processed_response = requests.get("http://app.sigrow.com/api/v2/camera/{}/shot/{}/processed".format(
                                                selected_camera, selected_shot["id"]), 
                                               headers={
                                                "X-API-Key": X_API_KEY
                                                }).json()
        
        # Filter for image
        thermal_plus_high_res = list(filter(get_temperature_plus_high_res_processed, shot_processed_response["processed"]))[0]
        
        # Create filename from original Sigrow filename
        fname=path+thermal_plus_high_res['url'].split('/')[-1]
        
        # Get image itself
        terbigpic=requests.get(thermal_plus_high_res["url"]).content
        
        # Save image to a file
        with open(fname, 'wb') as out_file:
            out_file.write(terbigpic)